## Workflows

1. Update config.yaml –                                Yes
2. Update secrets.yaml [Optional] –                    No
3. Update params.yaml –                                Yes
4. Update the entity –                                 Yes
5. Update the configuration manager in src config –    Yes
6. Update the components –                             Yes
7. Update the pipeline –                               Yes
8. Update the main.py –                                No
9. Update the dvc.yaml –                               No


In [1]:
import os

In [2]:
%pwd

'd:\\mlops\\deep-lerning_project\\end_to_end_chest_classification_with_mlflow_and_dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlops\\deep-lerning_project\\end_to_end_chest_classification_with_mlflow_and_dvc'

## 1.Update config.yaml

### Write code in config.yaml file :

```yaml
artifacts_root: artifacts

prepare_base_model:
  root_dir: artifacts/prepare_base_model
  base_model_path: artifacts/prepare_base_model/base_model.h5
  updated_base_model_path: artifacts/prepare_base_model/base_model_updated.h5

## 3.Update params.yaml

### Write Code in params.ymal

```python
AUGMENTATION = True
IMAGE_SIZE = [224, 224, 3]  # as per VGG 16 model
BATCH_SIZE = 16
INCLUDE_TOP = False
EPOCHS = 1
CLASSES = 2
WEIGHTS = 'imagenet'
LEARNING_RATE = 0.01


## 4.Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

## 5.Update the configuration manager in src config

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

## 6.Update the components

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

## 7.Update the pipeline 

In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-03-17 13:52:56,974: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-17 13:52:56,979: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-17 13:52:56,979: INFO: common: created directory at: artifacts]
[2025-03-17 13:52:56,983: INFO: common: created directory at: artifacts/prepare_base_model]


58889256/58889256 [==============================] - 41s 1us/step
[2025-03-17 13:53:39,426: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     738